In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np

import chart_studio.plotly as py	
import plotly.graph_objs as go
from plotly import subplots
import plotly.express as px
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot, plot
import plotly.express as px

In [9]:
log_path = 'logs'
batch_selection_methods = [o for o in os.listdir(log_path) if os.path.isdir(os.path.join(log_path,o))]
files = [[os.path.join(log_path, o, f) for f in os.listdir(os.path.join(log_path, o)) if os.path.isfile(os.path.join(log_path, o, f))] for o in batch_selection_methods]

In [10]:
files = np.array(files)

In [11]:
files.shape

(3, 7)

In [12]:
plot_matrix = np.empty((files.T.shape), dtype=object)
for i in range(files.shape[0]):
    for j in range(files.shape[1]):
        f = files[i][j]
        data = pd.read_csv(f)
        plot_matrix[j][i] = [data, f.split('/')[2].split('.')[0], f.split('/')[1]]


In [13]:
color_list = px.colors.qualitative.T10

In [14]:
for i in range(plot_matrix.shape[0]):
    fig = subplots.make_subplots(rows=3, cols=2, vertical_spacing=0.06, horizontal_spacing=0.02, 
                            subplot_titles=("Time-based Loss", "Epoch-based Loss", 
                                            "Time-based Training Metrics", "Epoch-based Training Metrics",
                                            "Time-based Validation Metrics" , "Epoch-based Validation Metrics"))
        
    for j in range(plot_matrix.shape[1]):
        plot_data = plot_matrix[i][j][0]
        name = plot_matrix[i][j][1].upper()
        selection = plot_matrix[i][j][2].title()
        color = color_list[j]
        time_based_loss_plot = go.Scatter(x=plot_data['Time'], y=plot_data['Loss'], name = selection + ' Loss Time', mode='lines', line_color=color)
        time_based_metrics_plot = go.Scatter(x=plot_data['Time'], y=plot_data['Metrics'], name = selection + ' Metrics Time', mode='lines', line_color=color)
        time_based_vl_plot = go.Scatter(x=plot_data['Time'], y=plot_data['Validation'], name =  selection + ' Validation Time', mode='lines', line_color=color)

        epoch_based_loss_plot = go.Scatter(x=plot_data['Epoch'], y=plot_data['Loss'], name = selection + ' Loss Epoch', mode='lines', line_color=color)
        epoch_based_metrics_plot = go.Scatter(x=plot_data['Epoch'], y=plot_data['Metrics'], name = selection + ' Metrics Epoch', mode='lines', line_color=color)
        epoch_based_vl_plot = go.Scatter(x=plot_data['Epoch'], y=plot_data['Validation'], name = selection + ' Validation Epoch', mode='lines',line_color=color)

        fig.append_trace(time_based_loss_plot, 1, 1)
        fig.append_trace(time_based_metrics_plot, 2, 1)
        fig.append_trace(time_based_vl_plot, 3, 1)

        fig.append_trace(epoch_based_loss_plot, 1, 2)
        fig.append_trace(epoch_based_metrics_plot, 2, 2)
        fig.append_trace(epoch_based_vl_plot, 3, 2)

        fig.update_layout(title=go.layout.Title(text = name))

    plot(fig, filename='plots/' + str(i) + '.html')

In [ ]:
for i in range(plot_matrix.shape[0]):
    for j in range(plot_matrix.shape[1]):
        plot_data = plot_matrix[i][j][0]
        name = plot_matrix[i][j][1].upper()
        selection = plot_matrix[i][j][2].title()